In [2]:
import cobra
from cobra import Model, Reaction, Metabolite
import numpy as np
import pandas as pd
import random
from os.path import join
from itertools import chain, permutations, product





In [3]:
fap_ecm = pd.read_csv('WedmarkModel\exchange-enumeration\_results\ecm\_fap.csv', index_col=0)
srb_ecm = pd.read_csv('WedmarkModel\exchange-enumeration\_results\ecm\srb.csv', index_col=0)
syn_ecm = pd.read_csv('WedmarkModel\exchange-enumeration\_results\ecm\syn.csv', index_col=0)

ecms = [fap_ecm, srb_ecm, syn_ecm]

print(srb_ecm)

   M_SO4ex_gen_srb  M_H2Sex_gen_srb  M_ATPex_srb  M_bm_srb  M_CO2ex_gen_srb  \
0        -1.000000         1.000000          0.0       0.0          0.00000   
1        -2.000000         2.000000          1.5       0.0          0.00000   
2        -1.000000         1.000000          0.0       0.0          2.80000   
3      -328.893333       328.893333          0.0       3.0          0.00000   
4      -492.531875       492.531875          0.0       3.0        981.83125   
5      -160.668000       160.668000          0.0       1.5        -22.67200   
6      -107.112000       107.112000          0.0       1.0        -45.99600   
7        -1.000000         1.000000          0.0       0.0          2.00000   
8        -1.000000         1.000000          0.5       0.0          2.00000   

   M_NH3ex_gen_srb  M_H2pool_gen_srb  M_ac_pool_srb  
0           0.0000         -4.000000       0.000000  
1           0.0000         -8.000000       0.000000  
2           0.0000          1.600000      -1.40

In [4]:
def calculate_efficiency(row, product_col, substrate_col):
    if row[substrate_col] != 0:
        return row[product_col] / abs(row[substrate_col])
    return 0

In [5]:
fap_ecm['bm_efficiency']  = fap_ecm.apply(lambda row: calculate_efficiency(row, 'M_bm_fap', 'M_hv_gen_fap'), axis=1)
srb_ecm['bm_efficiency'] = srb_ecm.apply(lambda row: calculate_efficiency(row, 'M_bm_srb', 'M_H2pool_gen_srb'), axis=1)
syn_ecm['bm_efficiency'] = syn_ecm.apply(lambda row: calculate_efficiency(row, 'M_bm_syn', 'M_hv_gen_syn'), axis=1)

fap_ecm['atp_efficiency'] = fap_ecm.apply(lambda row: calculate_efficiency(row, 'M_ATPex_fap', 'M_hv_gen_fap'), axis=1)
srb_ecm['co2_efficiency'] = srb_ecm.apply(lambda row: calculate_efficiency(row, 'M_CO2ex_gen_srb', 'M_SO4ex_gen_srb'), axis=1)
syn_ecm['o2_efficiency'] = syn_ecm.apply(lambda row: calculate_efficiency(row, 'M_O2expool_gen_syn', 'M_hv_gen_syn'), axis=1)


print("Top 5 Biomass Efficient EMs for FAP:")
print(fap_ecm.nlargest(5, 'bm_efficiency')[['bm_efficiency', 'M_bm_fap', 'M_hv_gen_fap']])

print("\nTop 5 ATP Efficient EMs for FAP:")
print(fap_ecm.nlargest(5, 'atp_efficiency')[['atp_efficiency', 'M_ATPex_fap', 'M_hv_gen_fap']])

print("\nTop 5 Biomass Efficient EMs for SRB:")
print(srb_ecm.nlargest(5, 'bm_efficiency')[['bm_efficiency', 'M_bm_srb', 'M_H2pool_gen_srb']])

print("\nTop 5 CO2 Efficient EMs for SRB:")
print(srb_ecm.nlargest(5, 'co2_efficiency')[['co2_efficiency', 'M_CO2ex_gen_srb', 'M_SO4ex_gen_srb']])

print("\nTop 5 Biomass Efficient EMs for SYN:")
print(syn_ecm.nlargest(5, 'bm_efficiency')[['bm_efficiency', 'M_bm_syn', 'M_hv_gen_syn']])

print("\nTop 5 O2 Efficient EMs for SYN:")
print(syn_ecm.nlargest(5, 'o2_efficiency')[['o2_efficiency', 'M_O2expool_gen_syn', 'M_hv_gen_syn']])



Top 5 Biomass Efficient EMs for FAP:
    bm_efficiency  M_bm_fap  M_hv_gen_fap
58       0.024613  0.079917     -3.246970
77       0.008361  0.079917     -9.558806
30       0.008197  0.079917     -9.750073
45       0.008095  0.079917     -9.872026
39       0.007966  0.079917    -10.032872

Top 5 ATP Efficient EMs for FAP:
    atp_efficiency  M_ATPex_fap  M_hv_gen_fap
44             0.5          0.5     -1.000000
0              0.0          0.0    -12.000000
1              0.0          0.0    -11.130611
2              0.0          5.0      0.000000
3              0.0          0.0      0.000000

Top 5 Biomass Efficient EMs for SRB:
   bm_efficiency  M_bm_srb  M_H2pool_gen_srb
3       0.002292       3.0      -1309.108333
5       0.002190       1.5       -684.783500
6       0.001929       1.0       -518.285000
0       0.000000       0.0         -4.000000
1       0.000000       0.0         -8.000000

Top 5 CO2 Efficient EMs for SRB:
   co2_efficiency  M_CO2ex_gen_srb  M_SO4ex_gen_srb
2      

In [6]:
selected_ecms = pd.concat([fap_ecm.nlargest(3, 'bm_efficiency'), fap_ecm.nlargest(3, 'atp_efficiency'), srb_ecm.nlargest(3, 'bm_efficiency'), srb_ecm.nlargest(3, 'co2_efficiency'), syn_ecm.nlargest(3, 'bm_efficiency'), syn_ecm.nlargest(4, 'o2_efficiency')], sort = False, ignore_index=True).fillna(0)
selected_ecms = selected_ecms.drop('bm_efficiency', axis=1)
selected_ecms = selected_ecms.drop('atp_efficiency', axis=1)
selected_ecms = selected_ecms.drop('co2_efficiency', axis=1)
selected_ecms = selected_ecms.drop('o2_efficiency', axis=1)

selected_ecms_syn = pd.concat([syn_ecm.nlargest(3,'bm_efficiency'),syn_ecm.nlargest(4,'o2_efficiency')], sort = False, ignore_index=True).fillna(0)
selected_ecms_syn = selected_ecms_syn.drop('bm_efficiency', axis=1)
selected_ecms_syn = selected_ecms_syn.drop('o2_efficiency', axis=1)
for met in selected_ecms.columns:
    #if 'ex' not in met:
        print(met)
#print(selected_ecms.columns)

M_ATPex_fap
M_CO2ex_gen_fap
M_H2pool_gen_fap
M_NH3ex_gen_fap
M_bm_fap
M_hv_gen_fap
M_glycpool_gen_fap
M_O2expool_gen_fap
M_PHB_fap
M_ac_pool_fap
M_SO4ex_gen_srb
M_H2Sex_gen_srb
M_ATPex_srb
M_bm_srb
M_CO2ex_gen_srb
M_NH3ex_gen_srb
M_H2pool_gen_srb
M_ac_pool_srb
M_polyglcex_gen_syn
M_CO2ex_gen_syn
M_NH3_syn
M_NH3ex_gen_syn
M_ATPex_syn
M_bm_syn
M_hv_gen_syn
M_O2expool_gen_syn
M_glycpool_gen_syn
M_glyox_syn
M_ac_pool_syn


In [7]:
def getNormalizedData2(ECM, T):

    sum_abs_coef = abs(ECM).sum(axis=1)#np.sum(np.abs(ECM), axis=1)
    normalized_ECM = ECM.div(sum_abs_coef, axis=0)

    return normalized_ECM

In [8]:

# Function to create a reaction from an ECM
def create_reaction_from_ecm(ecm, index, model):
    reaction = Reaction(f'ECM_{index}')
    reaction.name = f'Elementary Mode {index}'
    reaction.lower_bound = 0  # Assuming irreversible reactions
    reaction.upper_bound = 1000  # You might want to adjust this

    for metabolite, coefficient in ecm.items():
        if coefficient != 0.0:
            if metabolite not in model.metabolites:
                met = Metabolite(metabolite, compartment='c')
                model.add_metabolites([met])
                if 'bm_fap' in met.id:
                    bm_fap = met
                if 'bm_srb' in met.id:
                    bm_srb = met
                if 'bm_syn' in met.id:
                    bm_syn = met
            else:
                met = model.metabolites.get_by_id(metabolite)
            reaction.add_metabolites({met: round(coefficient,3)})
        
    return reaction

In [9]:
model = Model('nested_model_fromscratch')
model_syn = Model('nested_syn')


ecm_syn = getNormalizedData2(selected_ecms_syn, "293.15K")
ecm = getNormalizedData2(selected_ecms, "293.15K")    

for index, ecm in selected_ecms.iterrows():
    reaction = create_reaction_from_ecm(ecm, index, model)
    model.add_reactions([reaction])

for index, ecm in selected_ecms_syn.iterrows():
    reaction = create_reaction_from_ecm(ecm, index, model_syn)
    model_syn.add_reactions([reaction])

for reac in model_syn.reactions:
    print(reac)

Restricted license - for non-production use only - expires 2025-11-24
ECM_0: 1.838 M_CO2ex_gen_syn + 0.5 M_NH3_syn + 15.724 M_hv_gen_syn --> 1.795 M_O2expool_gen_syn + 0.04 M_bm_syn
ECM_1: M_CO2ex_gen_syn + 7.125 M_hv_gen_syn --> 0.5 M_O2expool_gen_syn + 0.5 M_glyox_syn
ECM_2: M_CO2ex_gen_syn + 8.0 M_hv_gen_syn --> M_O2expool_gen_syn + 0.5 M_ac_pool_syn
ECM_3: M_CO2ex_gen_syn + 8.0 M_hv_gen_syn --> M_O2expool_gen_syn + 0.5 M_ac_pool_syn
ECM_4: 3.0 M_CO2ex_gen_syn + 24.5 M_hv_gen_syn --> 3.0 M_O2expool_gen_syn + 0.5 M_polyglcex_gen_syn
ECM_5: 1.838 M_CO2ex_gen_syn + 0.5 M_NH3_syn + 15.724 M_hv_gen_syn --> 1.795 M_O2expool_gen_syn + 0.04 M_bm_syn
ECM_6: M_CO2ex_gen_syn + 10.25 M_hv_gen_syn --> 0.75 M_O2expool_gen_syn + 0.5 M_glycpool_gen_syn


In [10]:
for met_id in model_syn.metabolites:
    if 'bm' in met_id.id:
            model_syn.add_boundary(model_syn.metabolites.get_by_id(met_id.id), type="demand")
    else:
        met = model_syn.metabolites.get_by_id(met_id.id)
        met.compartment = 'e'
        model_syn.add_boundary(model_syn.metabolites.get_by_id(met_id.id), type="exchange")

for reac in model_syn.reactions:
    #for met in reac.metabolites:
        if 'DM_M_bm' in reac.id:
            print(reac)
            objective_id = str(reac.id)
            reac.objective_coefficient = 1
            biomass_reaction = model_syn.reactions.get_by_id(reac.id)  
            model_syn.objective = biomass_reaction
            print(model_syn.objective)
            break
            

DM_M_bm_syn: M_bm_syn --> 
Maximize
1.0*DM_M_bm_syn - 1.0*DM_M_bm_syn_reverse_688e6


In [11]:
for met in model.reactions:
    print(met)
cobra.io.write_sbml_model(model_syn, 'nested_model_syn.xml')

syn = cobra.io.read_sbml_model('syn.xml')



ECM_0: M_NH3ex_gen_fap + 0.574 M_O2expool_gen_fap + 2.776 M_glycpool_gen_fap + 3.247 M_hv_gen_fap --> 1.876 M_CO2ex_gen_fap + 0.08 M_bm_fap
ECM_1: M_NH3ex_gen_fap + 2.776 M_glycpool_gen_fap + 9.559 M_hv_gen_fap --> 1.876 M_CO2ex_gen_fap + 1.148 M_H2pool_gen_fap + 0.08 M_bm_fap
ECM_2: M_NH3ex_gen_fap + 2.393 M_glycpool_gen_fap + 9.75 M_hv_gen_fap --> 1.11 M_CO2ex_gen_fap + 0.08 M_bm_fap
ECM_3: M_hv_gen_fap --> 0.5 M_ATPex_fap
ECM_4: 3.0 M_ac_pool_fap + 0.5 M_glycpool_gen_fap + 12.0 M_hv_gen_fap --> M_CO2ex_gen_fap + 1.5 M_PHB_fap
ECM_5: M_NH3ex_gen_fap + 0.524 M_ac_pool_fap + 1.728 M_glycpool_gen_fap + 11.131 M_hv_gen_fap --> 0.828 M_CO2ex_gen_fap + 0.1 M_H2pool_gen_fap + 0.08 M_bm_fap
ECM_6: 1309.108 M_H2pool_gen_srb + 37.539 M_NH3ex_gen_srb + 328.893 M_SO4ex_gen_srb + 68.994 M_ac_pool_srb --> 328.893 M_H2Sex_gen_srb + 3.0 M_bm_srb
ECM_7: 22.672 M_CO2ex_gen_srb + 684.784 M_H2pool_gen_srb + 18.77 M_NH3ex_gen_srb + 160.668 M_SO4ex_gen_srb + 23.161 M_ac_pool_srb --> 160.668 M_H2Sex_gen_sr

In [12]:
additional_reaction = {
'vlv_glyc_fap': {'metabolites': {'M_glycpool_gen_syn': -0.5, 'M_glycpool_gen_fap': 0.5}},
'vlv_glyc_srb': {'metabolites': {'M_glycpool_gen_syn': -0.5, 'M_glycpool_gen_srb': 0.5}},
'vlv_ac_fap': {'metabolites': {'M_ac_pool_syn': -0.5, 'M_ac_pool_fap': 0.5}},
'vlv_ac_srb': {'metabolites': {'M_ac_pool_syn': -0.5, 'M_ac_pool_syn':0.5}}
}
glyc_syn = model.metabolites.get_by_id('M_glycpool_gen_syn')
glyc_fap = model.metabolites.get_by_id('M_glycpool_gen_fap')
ac_syn = model.metabolites.get_by_id('M_ac_pool_syn')
ac_fap = model.metabolites.get_by_id('M_ac_pool_fap')
ac_srb = model.metabolites.get_by_id('M_ac_pool_srb')





vlv_glyc_fap = Reaction('vlv_glyc_fap')
vlv_glyc_fap.id = 'vlv_glyc_fap'
vlv_glyc_fap.lower_bound = -1000
vlv_glyc_fap.upper_bound = 1000
vlv_glyc_fap.add_metabolites({glyc_syn: -0.5,glyc_fap: 0.5})
model.add_reactions([vlv_glyc_fap])

vlv_ac_fap = Reaction('vlv_ac_fap')
vlv_ac_fap.id = 'vlv_ac_fap'
vlv_ac_fap.lower_bound = -1000
vlv_ac_fap.upper_bound = 1000
vlv_ac_fap.reversibility = True
vlv_ac_fap.add_metabolites({ac_syn: -0.5,ac_fap: 0.5})
model.add_reactions([vlv_ac_fap])

vlv_ac_srb = Reaction('vlv_ac_srb')
vlv_ac_srb.id = 'vlv_ac_srb'
vlv_ac_srb.lower_bound = -1000
vlv_ac_srb.upper_bound = 1000
vlv_ac_srb.reversibility = True
vlv_ac_srb.add_metabolites({ac_syn: -0.5,ac_srb: 0.5})
model.add_reactions([vlv_ac_srb])


C:\Users\Jasovsju\anaconda3\Lib\site-packages\cobra\core\reaction.py:810: UserWarning: Setting reaction reversibility is ignored
  warn("Setting reaction reversibility is ignored")


In [13]:
bm_fap_srb_syn = Metabolite('bm_fap_srb_syn', compartment='c')
bm = Metabolite('bm', compartment='c')
model.add_metabolites([bm_fap_srb_syn])
bm_obj = Reaction('bm_obj')
bm1 = Reaction('bm1')
bm2 = Reaction('bm2')
bm3 = Reaction('bm3')

#bm.id = 'bm'
#bm.name = 'Community Biomass'
bm_fap = model.metabolites.get_by_id('M_bm_fap')
bm_srb = model.metabolites.get_by_id('M_bm_srb')
bm_syn = model.metabolites.get_by_id('M_bm_syn')
bm_obj.add_metabolites({bm_fap: -1, bm_srb: -1, bm_syn: -1, bm: 1})
bm1.add_metabolites({bm_fap: -1, bm_fap_srb_syn: 1})
bm2.add_metabolites({bm_srb: -1, bm_fap_srb_syn: 1})
bm3.add_metabolites({bm_syn: -1, bm_fap_srb_syn: 1})
model.add_reactions([bm_obj, bm1, bm2, bm3])
print(bm)

objective_id = str(bm_obj.id)
bm_obj.objective_coefficient = 1
biomass_reaction = model.reactions.get_by_id('bm_obj')  
model.objective = biomass_reaction
print(model.objective)



bm
Maximize
1.0*bm_obj - 1.0*bm_obj_reverse_b59b4


In [14]:
for met in model.metabolites:
    meta = model.metabolites.get_by_id(met.id)
    print(met.id)

M_CO2ex_gen_fap
M_NH3ex_gen_fap
M_bm_fap
M_hv_gen_fap
M_glycpool_gen_fap
M_O2expool_gen_fap
M_H2pool_gen_fap
M_ATPex_fap
M_PHB_fap
M_ac_pool_fap
M_SO4ex_gen_srb
M_H2Sex_gen_srb
M_bm_srb
M_NH3ex_gen_srb
M_H2pool_gen_srb
M_ac_pool_srb
M_CO2ex_gen_srb
M_ATPex_srb
M_CO2ex_gen_syn
M_NH3_syn
M_bm_syn
M_hv_gen_syn
M_O2expool_gen_syn
M_glyox_syn
M_ac_pool_syn
M_polyglcex_gen_syn
M_glycpool_gen_syn
bm_fap_srb_syn
bm


In [15]:
       
demand_metabolites = [
     "M_PHB_fap", "M_ATPex_srb",  "M_ATPex_fap", 'bm_fap_srb_syn', "bm"#, 'M_bm_fap', 'M_bm_srb', 'M_bm_syn'#"M_ATPex_syn",
]
sink_metabolites = [
     "M_hv_gen_fap", "M_hv_gen_syn", "M_polyglcex_gen_syn"#, "hv_gen", 
]
exchange_metabolites = [
    'M_CO2ex_gen_fap','M_CO2ex_gen_srb','M_CO2ex_gen_syn', 'M_H2Sex_gen_srb', 'M_NH3ex_gen_fap','M_NH3ex_gen_srb','M_NH3_syn',
    'M_O2expool_gen_fap','M_O2expool_gen_syn', 'M_SO4ex_gen_srb', 'M_H2pool_gen_fap', 'M_H2pool_gen_srb'#, "M_hv_gen_fap", "M_hv_gen_syn", "M_glycpool_gen_fap"#, 'M_bm_fap', 'M_bm_srb', 'M_bm_syn'
]

# Austauschreaktionen für alle externen Metaboliten hinzufügen
for met_id in demand_metabolites:
    model.add_boundary(model.metabolites.get_by_id(met_id), type="demand")

for met_id in sink_metabolites:
    model.add_boundary(model.metabolites.get_by_id(met_id), type="sink")
    met = model.metabolites.get_by_id(met_id)
    #met.compartment = 'e'
    
for met_id in exchange_metabolites:
    met = model.metabolites.get_by_id(met_id)
    met.compartment = 'e'
    model.add_boundary(model.metabolites.get_by_id(met_id), type="exchange")
    

In [ ]:
cobra.io.write_sbml_model(model, 'nested_model.xml')
